# Redis Assignment

The purpose of this task is to create the cookie syncronization service between two companies with the help of Redis.

In [1]:
import redis
import time
from datetime import timedelta

class LimitExceededException(Exception):
    pass

max_rps = 15  # requests per second

# Basic database settings. Don't change this line!
r = redis.StrictRedis(host='localhost', port=6381, db=0) 

In [2]:
def save_sync(r, uid, partner_id, partner_uid):
    """
        Set the values for the pairs <partner_id, uid> and <partner_id, partner_uid>
        Don't forget to set the ttls which you defined in the function set_ttls
    """
    pipe = r.pipeline()
    pipe.hmset(str(partner_id), {'user:'+uid:partner_uid})
    pipe.hmset(str(partner_id), {'partner:'+partner_uid:uid})
    
    set_ttls(pipe, {partner_id : 3600})
    pipe.execute()
    
def get_partner_uid(r, uid, partner_id):
    """Get the partner id by the pair (uid, partner id)
    
    Args:
        r (redis.StrictRedis): redis instance
        uid (str): our uid
        partner_id (str): id of the partner
        
    Examples:
        >>> get_partner_uid(r, 'e5a370cc-6bdc-43ae-baaa-8fd4531847f7', 12)
    """
    restrict_function('get_partner_uid')
    return r.hget(str(partner_id), str('user:'+uid))



def get_uid(r, partner_id, partner_uid):
    """Get the uid by the pair (partner id, partner uid)
    
    Args:
        r (redis.StrictRedis): redis instance
        partner_id (int): id of partner
        partner_uid (str): uid of partner
        
    Examples:
        >>> get_uid(r, 12, '25b6e9a6-fca8-427c-94df-2577e62b2bf0')
    """
    restrict_function('get_uid')
    return r.hget(str(partner_id), str('partner:'+partner_uid))


def set_ttls(r, ttls):
    """Set the ttl by partner id
       
    Args:
        r (redis.StrictRedis): redis instance
        ttls (dict): dictionary of pairs <partner_id, ttl>
        
    Examples:
        >>> set_ttls(r, {12: 5, 3: 0.25})
    """
    for key, seconds in ttls.items():
        miliseconds = int(seconds*1000)
        r.pexpire(str(key), miliseconds)

def restrict_function(func_name):
    pipe = r.pipeline()
    if not r.get(func_name):
        pipe.set(func_name, 0)   
        set_ttls(pipe, {func_name : 1})
    
    pipe.incrby(func_name, 1)
    pipe.execute()
    if(int(r.get(func_name))>max_rps):
        raise LimitExceededException

There are some basic checks for your function

In [3]:
# Usage examples


save_sync(r, 'e5a370cc-6bdc-43ae-baaa-8fd4531847f7', 12, '25b6e9a6-fca8-427c-94df-2577e62b2bf0')
set_ttls(r, {12: 20, 3: 20})

# Some basic checks

assert 'e5a370cc-6bdc-43ae-baaa-8fd4531847f7' == get_uid(r, 12, '25b6e9a6-fca8-427c-94df-2577e62b2bf0').decode('utf-8')

assert '25b6e9a6-fca8-427c-94df-2577e62b2bf0' == get_partner_uid(r, 'e5a370cc-6bdc-43ae-baaa-8fd4531847f7', 12).decode('utf-8')


In [4]:
# for i in range(100):
#     try:
#         print('da')
#         print(get_uid(r, 12, '25b6e9a6-fca8-427c-94df-2577e62b2bf0').decode('utf-8'))
#     except : 
#         pass

Congratulations! You have completed the assignment! Now you can submit it to the system and get your results!